In [ ]:
# import pickle
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from paths import path_CESM
from paths import path_ocn_ctrl, path_ocn_ctrl_rect
from paths import path_ocn_rcp,  path_ocn_rcp_rect
from paths import CESM_filename, file_ex_ocn_rect
from timeseries import IterateOutputCESM, yrly_avg_nc

In [ ]:
%%time
yrly_avg_nc('ocn', 'ctrl', ['SHF'], test=False)
yrly_avg_nc('ocn', 'rcp', ['SHF'], test=False)

In [ ]:
%%time
yrly_avg_nc('atm', 'ctrl', ['FSNT', 'FLNT'], test=False)
yrly_avg_nc('atm', 'rcp', ['FSNT', 'FLNT'], test=False)

In [ ]:
ds = xr.open_dataset(file_ex_ocn_rect, decode_times=False)

In [ ]:
ds.SHF[:,:].plot(vmin=-300, vmax=300, cmap='RdBu_r')

In [ ]:
ds = xr.open_dataset('/projects/0/samoc/andre/CESM/rcp/ocn_yrly_None_2000.nc', decode_times=False)

In [ ]:
nc_file = f'/projects/0/prace_imau/prace_2013081679/rene/CESM/OceanHeatContent/Global_0.1/OHC_0250-01_All.nc'
xd_rene = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
xd_rene

In [ ]:
plt.imshow(xd_rene.OHC[0,0,:,:],vmin=0,vmax=1.035*1.6e-3)
plt.colorbar()

In [ ]:
xd_rene.OHC[0,-1,:,:].sum().item()

In [ ]:
xd_rene.OHC[0,:,:,:].where(xd_rene.OHC[0,:,:,:] < 1).sum()